# Inventário

Deve-se executar as seis primeiras células para carregar funções utilizadas para rodar o script

In [1]:
import json
import os
from openpyxl import load_workbook
from datetime import timedelta
import random
from copy import deepcopy
from django.contrib.gis.geos import LineString, Point
from django.contrib.admin.utils import flatten
import math
import datetime
import locale
from tqdm.notebook import tqdm
from PIL import Image
from io import BytesIO
from simple_history.utils import bulk_create_with_history
from django.contrib.gis.geos import Point, LineString as DjangoLineString
from helpers.histories import bulk_update_with_history
import pytz

In [2]:
from helpers.route_maker import dic_to_ordered_list, unequal_point_pairs

In [3]:
from django.contrib.gis.geos import Point, LineString as DjangoLineString
from django.contrib.gis.db.models.functions import Distance
from turfpy.misc import nearest_point_on_line
from turfpy.measurement import length
from geojson import LineString, loads
from helpers.histories import bulk_update_with_history
from tqdm.notebook import tqdm

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [4]:
def km_to_coordinates(road, km):

    road_marks = dic_to_ordered_list(road.marks)
    selected_pair = None

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] > km and pair[1]["km"] < km:
            selected_pair = pair
            break
        elif pair[1]["km"] > km and pair[0]["km"] < km:
            selected_pair = pair
            break
        elif pair[0]["km"] == km:
            return Point(pair[0]["point"]["coordinates"]), road
        elif pair[1]["km"] == km:
            return Point(pair[1]["point"]["coordinates"]), road

    # Calculate total length
    start_km = min([selected_pair[0]["km"], selected_pair[1]["km"]])
    end_km = max([selected_pair[0]["km"], selected_pair[1]["km"]])
    segment_mark_length = end_km - start_km
    distance_from_min = km - start_km
    # Check if km is decreasing
    invert_km = selected_pair[1]["km"] <= selected_pair[0]["km"]

    # Cut segment
    start_key = min((selected_pair[0]["index"], selected_pair[1]["index"]))
    end_key = max((selected_pair[0]["index"], selected_pair[1]["index"]))

    points = []
    for key in range(start_key, end_key + 1):
        points.append(
            Point(road.path[key][0], road.path[key][1], road.path[key][2])
        )

    segment = LineString(points, srid=4326)
    # Apply km inversion correction
    if invert_km:
        segment_distance = (
            1 - (distance_from_min / segment_mark_length)
        ) * segment.length
    else:
        segment_distance = (
            distance_from_min / segment_mark_length
        ) * segment.length
    # Find point
    point = segment.interpolate(segment_distance)

    return Point(point.x, point.y), road

In [5]:
def check_valid_road(road, km):

    road_marks = dic_to_ordered_list(road.marks)

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] >= km and pair[1]["km"] <= km:
            return True
        elif pair[1]["km"] >= km and pair[0]["km"] <= km:
            return True

    return False

In [6]:
def get_road_coordinates(road_name, km, direction, company):
    road_set = Road.objects.filter(
        name__contains=road_name, direction=int(direction), company=company
    )

    # If roads in specified direction are not found, search road only
    # by name and order then by direction
    road_set_generic = Road.objects.filter(
        name__contains=road_name, company=company
    ).order_by("direction")

    # Check if KM range in road_set
#     print("Searching KM on direction...")
    valid = False
    for road in road_set:
        if check_valid_road(road, km):
#             print("Found KM on Road {}".format(road))
            valid = True
            break

    if not valid:
#         print("Searching KM without direction...")
        for road in road_set_generic:
            if check_valid_road(road, km):
#                 print("Found KM on Road {}".format(road))
                valid = True
                break

    if not valid:
        return Point(0, 0), None

    try:
        return km_to_coordinates(road, km)
    except Exception:
        return Point(0, 0), None

In [7]:
def findIndex(flist, func):
    for i,v in enumerate(flist):
        if func(v): 
            return i
    return -1

In [46]:
def point_to_km(lng, lat, road):
    point = Point(lng, lat, srid=4326)
#     roads = Road.objects.filter(company=company).annotate(distance=Distance(point, 'path')).order_by('distance')
#     road = roads[0]
    path = loads(road.path.geojson)
    point_geojson = loads(point.geojson)
    road_marks = list(road.marks.values())
    snap_point = nearest_point_on_line(path, point_geojson)
    

    road_markers = sorted([a for a in road_marks if 'index' in a or ('index' in a and a['index'] == 0)], key=lambda x: int(x['key']))
#     road_markers = sorted([a for a in road_marks if a['index'] or a['index'] == 0], key=lambda x: int(x['key']))
    rm_index = findIndex(road_markers, lambda x: x['index'] > snap_point['properties']['index'])
    
    if (rm_index == -1):
        rm_index = len(road_markers) - 1;
    
    km_marker_start = road_markers[rm_index - 1]
    km_marker_end = road_markers[rm_index]
    
    if km_marker_start['index'] == road_markers[-1]['index']: 
        snap = road_markers[-1]
        km = snap['km']
    elif km_marker_end['index'] == road_markers[0]['index']:
        snap = road_markers[0]
        km = snap['km']
    else:
        test = path['coordinates'][km_marker_start['index']:km_marker_end['index'] + 1]
        marker_path = LineString(test)
        snap = nearest_point_on_line(marker_path, point_geojson)        
        ls_length = length(marker_path, units="km")
        km = ((snap['properties']['location'] / ls_length)        
            * (km_marker_end['km'] - km_marker_start['km'])
            + km_marker_start['km'])
    
    return road, road.name, point, round(km, 3)

In [29]:
from collections import defaultdict
from django.db.models.signals import *


class DisableSignals(object):
    def __init__(self, disabled_signals=None):
        self.stashed_signals = defaultdict(list)
        self.disabled_signals = disabled_signals or [
            pre_init, post_init,
            pre_save, post_save,
            pre_delete, post_delete,
            pre_migrate, post_migrate,
        ]

    def __enter__(self):
        for signal in self.disabled_signals:
            self.disconnect(signal)

    def __exit__(self, exc_type, exc_val, exc_tb):
        for signal in list(self.stashed_signals):
            self.reconnect(signal)

    def disconnect(self, signal):
        self.stashed_signals[signal] = signal.receivers
        signal.receivers = []

    def reconnect(self, signal):
        signal.receivers = self.stashed_signals.get(signal, [])
        del self.stashed_signals[signal]

In [50]:
def get_connected_reference(company, resource, key, value):
    select_options = company.custom_options[resource]['fields'][key]['selectOptions']['options']
    return next(a['value'] for a in select_options if a['name'].lower() == value.lower())

In [31]:
def get_sign(field, sign_field, occ, value):
    if sign_field == 'Cor':
        value=value.capitalize()
    elif sign_field == 'Tipo de Película':
        value=value
    values = occ.form_fields['fields']
    item_translation = flatten([item['innerFields'] for item in values if item['displayName']==field])
#     print(item_translation)
    mid_translation = flatten([item['selectOptions']['options'] for item in item_translation if item['displayName'] == sign_field])
#     print(mid_translation)
    final_translation = {item['name']: item['value'] for item in mid_translation}
#     print(final_translation)
    return final_translation[value]

In [32]:
def get_value(field, occ, value):
    values = occ.form_fields['fields']
    item_translation = flatten([item['selectOptions']['options'] for item in values if item['displayName']==field])
#     print(item_translation)

    final_translation = {item['name']: item['value'] for item in item_translation}
#     print(final_translation)
    return final_translation[value]

 Verificando se está no ambiente de produção

In [33]:
!cat ~/kartado-backend/.env

STAGE=PRODUCTION


Escolhendo nome do arquivo Excel para carregar o Inventário

In [34]:
filename='Sinalização Aérea ERS 122 - CORRIGIDO'

Escolhendo a aba da planilha do Excel

In [35]:
wb = load_workbook(filename + '.xlsx')
sheetname = wb.sheetnames[1]
ws = wb[sheetname]

Testando planilha carregada do Excel

In [36]:
print(ws['A1'].value)

km


Carregando os valores no script

In [37]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    if obj['Rodovia'] is not None:           
   
        obj['img'] = [a for a in ws._images if a.anchor._from.row == index]

        try:
            for a in obj['img']:
                a.name = 'CSG'
#             obj['img'][0].name='Verso'
        except:
            pass

        
    values.append(obj)

Teste de valores importados da planilha

In [24]:
len(values),values[0]

(197,
 {'km': None,
  'km final': None,
  'km de Projeto': 138.06493342427763,
  'km final de Projeto': None,
  'Latitude_Inicial': -29.667,
  'Longitude_Inicial': -51.2559,
  'Latitude_Final': None,
  'Longitude_Final': None,
  'Status': 'Identificado',
  'Equipe/Empreiteira': 'Cadastro incial',
  'Encontrado em': None,
  'Executado em': None,
  'Sentido': 'Crescente',
  'Pista': 'DUPLA',
  'Classe': 'Sinalização Aérea',
  'Lado': 'Direito',
  'Faixa': 'Não se aplica',
  'Rodovia': 'ERS-122',
  'Código': 'I-4',
  'Data da medição atual': datetime.datetime(2023, 3, 8, 0, 0),
  'Fabricante': None,
  'Data de fabricação': None,
  'Tipo de suporte': 'PRAÇA PEDÁGIO',
  'Estado do suporte': None,
  'Tipo de Película': 'I',
  'Cor_1': 'AZUL',
  'Leitura 1 (cd/lx/m2)_1': 2,
  'Leitura 2 (cd/lx/m2)_1': 2.5,
  'Leitura 3 (cd/lx/m2)_1': 2.6,
  'Leitura 4 (cd/lx/m2)_1': 4,
  'Leitura 5 (cd/lx/m2)_1': 3.9,
  'Média das leituras_1': 3,
  'Valor mínimo_1': 4,
  'Valor Residual_1': 2,
  'Cor_2': 'BRA

Definindo valores da importação

In [2]:
company = Company.objects.get(name='Caminhos da Serra Gaúcha')

#Classe Sinalização Aérea
occurrence_type=OccurrenceType.objects.get(uuid='122d1706-1bff-4960-9063-0b26f1059336')

user=User.objects.get(username='rlcs')
status=ServiceOrderActionStatus.objects.get(companies=company, name='Identificado')
# step=ApprovalStep.objects.get(approval_flow__company=company, name='Em Elaboração')
firm=Firm.objects.get(uuid='0edd2ecc-8d5e-4444-898b-0069a3e9c354')
sign_name='Retrorrefletância'
company,occurrence_type,firm

(<Company: 4edb7778-e350-4e77-8e1e-de5f87b1da7f: Caminhos da Serra Gaúcha>,
 <OccurrenceType: Sinalização Aérea - ['Caminhos da Serra Gaúcha']>,
 <Firm: [4edb7778-e350-4e77-8e1e-de5f87b1da7f: Caminhos da Serra Gaúcha] 0edd2ecc-8d5e-4444-898b-0069a3e9c354: Cadastro incial>)

In [52]:
for a in tqdm(values):
    a['Retrorrefletância'] = list(filter(None, [{'color': get_sign(sign_name,'Cor',occurrence_type,a['Cor_1']) if a['Cor_1'] is not None else None,'type_of_film': get_sign(sign_name,'Tipo de Película',occurrence_type,a['Tipo de Película']) if a['Tipo de Película'] is not None else None, 'first': a['Leitura 1 (cd/lx/m2)_1'], 'second': a['Leitura 2 (cd/lx/m2)_1'], 'third': a['Leitura 3 (cd/lx/m2)_1'], 'fourth': a['Leitura 4 (cd/lx/m2)_1'], 'fifth': a['Leitura 5 (cd/lx/m2)_1'], 'average': a['Média das leituras_1'], 'initial_value': a['Valor mínimo_1'], 'residual_value': a['Valor Residual_1']} if a['Cor_1'] is not None else None,
                                  {'color': get_sign(sign_name,'Cor',occurrence_type,a['Cor_2']) if a['Cor_2'] is not None else None,'type_of_film': get_sign(sign_name,'Tipo de Película',occurrence_type,a['Tipo de Película']) if a['Tipo de Película'] is not None else None, 'first': a['Leitura 1 (cd/lx/m2)_2'], 'second': a['Leitura 2 (cd/lx/m2)_2'], 'third': a['Leitura 3 (cd/lx/m2)_2'], 'fourth': a['Leitura 4 (cd/lx/m2)_2'], 'fifth': a['Leitura 5 (cd/lx/m2)_2'], 'average': a['Média das leituras_2'], 'initial_value': a['Valor mínimo_2'], 'residual_value': a['Valor Residual_2']} if a['Cor_2'] is not None else None,
                                  {'color': get_sign(sign_name,'Cor',occurrence_type,a['Cor_3']) if a['Cor_3'] is not None else None,'type_of_film': get_sign(sign_name,'Tipo de Película',occurrence_type,a['Tipo de Película']) if a['Tipo de Película'] is not None else None, 'first': a['Leitura 1 (cd/lx/m2)_3'], 'second': a['Leitura 2 (cd/lx/m2)_3'], 'third': a['Leitura 3 (cd/lx/m2)_3'], 'fourth': a['Leitura 4 (cd/lx/m2)_3'], 'fifth': a['Leitura 5 (cd/lx/m2)_3'], 'average': a['Média das leituras_3'], 'initial_value': a['Valor mínimo_3'], 'residual_value': a['Valor Residual_3']} if a['Cor_3'] is not None else None]))

  0%|          | 0/197 [00:00<?, ?it/s]

In [3]:
road=Road.objects.filter(company=company,name__icontains="122").annotate(distance=Distance(point, 'path')).order_by('distance').first()
road

NameError: name 'Distance' is not defined

In [47]:
road,_,point,km =point_to_km(float(values[3]['Longitude_Inicial']),float(values[3]['Latitude_Inicial']),road)
road,point,km,values[0]

(<Road: 2059: ERS-122>,
 38.194,
 {'km': None,
  'km final': None,
  'km de Projeto': 138.06493342427763,
  'km final de Projeto': None,
  'Latitude_Inicial': -29.667,
  'Longitude_Inicial': -51.2559,
  'Latitude_Final': None,
  'Longitude_Final': None,
  'Status': 'Identificado',
  'Equipe/Empreiteira': 'Cadastro incial',
  'Encontrado em': None,
  'Executado em': None,
  'Sentido': 'Crescente',
  'Pista': 'DUPLA',
  'Classe': 'Sinalização Aérea',
  'Lado': 'Direito',
  'Faixa': 'Não se aplica',
  'Rodovia': 'ERS-122',
  'Código': 'I-4',
  'Data da medição atual': datetime.datetime(2023, 3, 8, 0, 0),
  'Fabricante': None,
  'Data de fabricação': None,
  'Tipo de suporte': 'PRAÇA PEDÁGIO',
  'Estado do suporte': None,
  'Tipo de Película': 'I',
  'Cor_1': 'AZUL',
  'Leitura 1 (cd/lx/m2)_1': 2,
  'Leitura 2 (cd/lx/m2)_1': 2.5,
  'Leitura 3 (cd/lx/m2)_1': 2.6,
  'Leitura 4 (cd/lx/m2)_1': 4,
  'Leitura 5 (cd/lx/m2)_1': 3.9,
  'Média das leituras_1': 3,
  'Valor mínimo_1': 4,
  'Valor Resi

Criando itens de Inventário

In [54]:
objects=[]


for index, a in enumerate(tqdm(values)):

    long=str(a['Longitude_Inicial']).replace(',','.')
    lat=str(a['Latitude_Inicial']).replace(',','.')
    pista = 'Pista Dupla' if a['Pista'] == 'DUPLA' else 'Pista Simples'
    road,_,point,km =point_to_km(float(long),float(lat),road)
#     if a['Longitude_Final'] and a['Latitude_Final']:
#         long_end=str(a['Longitude_Inicial'])[:3]+'.'+str(a['Longitude_Inicial'])[3:]
#         lat_end=str(a['Latitude_Inicial'])[:3]+'.'+str(a['Latitude_Inicial'])[3:]
#         _,_,_,end_km =point_to_km(float(long_end),float(lat_end),company)
    if '122' not in road.name:
        print(f'Arquivo"{filename}" - Rodovia {road} - Km:{km} - Linha: {index+2}')
        break
    try:
        
        objects.append((Reporting(
                    company=company,
                    occurrence_type=occurrence_type,
                    lane=str(get_connected_reference(company, 'reporting', 'lane', pista)),
                    road=road,
                    road_name=road.name,
                    direction=str(get_connected_reference(company, 'reporting', 'direction', a['Sentido'])),
                    created_by=user,
#                     found_at=datetime.datetime('2023-05-08'),
                    km=float(km),
#                     end_km=float(end_km) if a['Longitude_Final'] is not None and a['Latitude_Final'] else None,
                    point=point,
                    firm=firm,
                    status=status,
                    form_data={
                        'code': a['Código'],
                        'side': get_value('Lado',occurrence_type,str(a['Lado']).capitalize().strip()) if a['Lado'] is not None else None,
                        'datemeasurement': a['Data da medição atual'].strftime('%Y-%m-%dT03:00:00.000Z') if a['Data da medição atual'] is not None and a['Data da medição atual'] != '' else None,
                        'manufacturer': a['Fabricante'] if a['Fabricante'] is not None else None,
                        'datemanufacture': datetime.datetime(int(a["Data de fabricação"]), 1, 1, 0, 0).strftime('%Y-%m-%dT03:00:00.000Z') if a['Data de fabricação'] is not None else None,
                        'typesupport': a['Tipo de suporte'] if a['Tipo de suporte'] is not None else None,
                        'statesupport': a['Estado do suporte'] if a['Estado do suporte'] is not None else None,
                        'retro': a['Retrorrefletância'] if a['Retrorrefletância'] is not None else None,
                        
                    }, 
                    ),a['img']))
    except Exception as e:

        print(f'Arquivo"{filename}" - Rodovia {road} - Km:{km} - Linha: {index+2} - {e}')
        print(objects.form_data)

  0%|          | 0/197 [00:00<?, ?it/s]

ValueError: could not convert string to float: 'None'

Salvando itens de Inventário no sistema e imprimindo os seriais

In [56]:
len(objects),objects[2][0].__dict__

(18,
 {'_state': <django.db.models.base.ModelState at 0x7f03af4accd0>,
  'uuid': UUID('56077283-5506-4d22-a994-421a7eb16f33'),
  'number': '',
  'company_id': UUID('4edb7778-e350-4e77-8e1e-de5f87b1da7f'),
  'road_name': 'ERS-122',
  'road_id': 2059,
  'km': 38.194,
  'end_km': None,
  'km_reference': None,
  'project_km': 0,
  'project_end_km': None,
  'point': <Point object at 0x7f03acd3a520>,
  'direction': '0',
  'lane': '1',
  'track': None,
  'branch': None,
  'address': {},
  'created_by_id': UUID('508a66f3-3f58-4c4e-bc9a-fbba6078c02d'),
  'firm_id': UUID('0edd2ecc-8d5e-4444-898b-0069a3e9c354'),
  'occurrence_type_id': UUID('122d1706-1bff-4960-9063-0b26f1059336'),
  'form_data': {'code': 'I-3',
   'side': '1',
   'datemeasurement': '2023-03-10T03:00:00.000Z',
   'manufacturer': None,
   'datemanufacture': None,
   'typesupport': 'PÓRTICO',
   'statesupport': 'BOM',
   'retro': [{'color': '2',
     'type_of_film': '1',
     'first': 3,
     'second': 4.5,
     'third': 6,
     'fo

In [57]:
for reporting, images in tqdm(objects):
    reporting.save()
    print(reporting.number)

  0%|          | 0/18 [00:00<?, ?it/s]

Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.18776
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.18777
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.18778
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.18779
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.18780
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.18781
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.18782
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.18783
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CS

Anexando as imagens

In [58]:
for reporting, images in tqdm(objects):
    for image in reversed(images):
        reporting_file = ReportingFile(
            created_by=user,
            reporting=reporting,
            description=image.name,
            km=reporting.km,
            point=reporting.point
        )
        reporting_file.save()
        image_io = BytesIO(image._data())
        im = Image.open(image_io)
        if im.mode in ("RGBA", "P"):
            im = im.convert("RGB")
        thumb_io = BytesIO()
        im.save(thumb_io, format='jpeg', quality=90)
        reporting_file.upload.save(image.name + '.jpeg', thumb_io)

  0%|          | 0/18 [00:00<?, ?it/s]

In [4]:
reps=Reporting.objects.filter(company=company,occurrence_type=occurrence_type,form_data__code="R-27")

In [6]:
for a in reps:
    print(a.number)

CSG-Inv-2023.02059
CSG-Inv-2023.02060
CSG-Inv-2023.02506
CSG-Inv-2023.02535
